# Cinématique inverse

Ce notebook explore la génération de mouvements pour un robot humanoïde en utilisant la **cinématique inverse**.  
L'exercice consiste à générer une trajectoire où :
- Le **centre de masse (CoM)** du robot suit une trajectoire à vitesse constante.
- Les **pieds** (gauche et droit) effectuent des **déplacements alternés**, comme lors d'une marche, également à vitesse constante.

Nous utiliserons pour cela une approche **cinématique**, sans résolution dynamique, et enchaînerons des tâches résolues via pseudo-inversion du jacobien (commande à vitesse).

Le mouvement sera composé de plusieurs **phases**, alternant appui gauche et appui droit.

In [ ]:
import gepetuto

## Chargement du robot et initialisation du visualiseur

Entête classique avec Pinocchio, example robot data et meshcat

In [ ]:
# %load tp3/generated/fake_walk_import
import time
from collections import namedtuple

import example_robot_data as robex
import numpy as np
import pinocchio as pin
from numpy.linalg import pinv

from schaeffler2025.meshcat_viewer_wrapper import MeshcatVisualizer

# --- Load robot model
robot = robex.load("talos")
model = robot.model
data = model.createData()
NQ = model.nq
NV = model.nv

# Open the viewer
viz = MeshcatVisualizer(robot)
viz.display(robot.q0)
feetIndexes = {
    "right": robot.model.getFrameId("right_sole_link"),
    "left": robot.model.getFrameId("left_sole_link"),
}


## Une marche approximative comme exemple

On définit une marche bipède comme une succession de phase de marche, chacune définissant:
- la durée de la phase
- le pied de support droit ou gauche (éventuellement *double* ou *none* dans des cas plus complexes)
- l'avancement du pied de vol par rapport à


In [ ]:
# %load tp3/generated/fake_walk_phases
# Structure to define a phase of locomotion
Phase = namedtuple("Phase", ["duration", "delta", "support"])

# Definition of the task
comVelocity = np.array([0.7, 0, 0])
stepDuration = 0.8
step = comVelocity[:2] * stepDuration * 2
phases = [
    Phase(stepDuration / 2, step / 2, "right"),
    Phase(stepDuration, step, "left"),
    Phase(stepDuration, step, "right"),
    Phase(stepDuration, step, "left"),
    Phase(stepDuration, step, "right"),
    Phase(stepDuration / 2, step / 2, "left"),
]
DT = 1e-2


## Cinématique inverse

À chaque instant de chaque phase, calculer la vitesse du robot à partir des éléments suivant:
- le COM avance à vitesse constante *comVelocity*
- le pied de support est à vitesse nulle
- le pied de vol avance à vitesse constante *phase.delta/phase.duration*

Vous pouvez calculer la cinématique directe avec la syntaxte suivante.

In [ ]:
q = robot.q0.copy()

# Effectue la plupart des calculs nécessaires
pin.computeAllTerms(model, data, q, np.zeros(NV))

# Jacobien d'un repère opérationel
J_rf = pin.getFrameJacobian(model, data, feetIndexes["right"], pin.LOCAL)

# Jacobien du COM
J_com = pin.jacobianCenterOfMass(model, data, q)


La pseudo-inverse est définie dans numpy.linalg.

In [ ]:
pinv(np.random.rand(3,5))

La cinématique inverse doit être implémenté dans une boucle correspondant au template suivant:

In [ ]:
q = robot.q0.copy()
for phase in phases:
    for t in np.arange(0, phase.duration, DT):
        # Pre-compute kinematics, jacobians, com etc
        pin.computeAllTerms(model, data, q, np.zeros(NV))

        vq = np.zeros(NV) ### TODO
        
        q = pin.integrate(model, q, vq * DT)
        viz.display(q)
        time.sleep(DT)


La solution est ci dessous ci nécessaire.

In [ ]:
%do_not_load tp3/generated/fake_walk_ik